In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
path = os.path.join('./gdrive/My Drive/idl_test')
os.chdir(path)

In [ ]:
!pip install adversarial-robustness-toolbox

In [4]:
"""
The script demonstrates a simple example of using ART with PyTorch. The example train a small model on the MNIST dataset
and creates adversarial examples using the Fast Gradient Sign Method. Here we use the ART classifier to train the model,
it would also be possible to provide a pretrained model to the ART classifier.
The parameters are chosen for reduced computational requirements of the script and not optimised for accuracy.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim.lr_scheduler as lr_scheduler

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import PyTorchClassifier
from art.utils import load_cifar10


In [5]:
class VGG16(nn.Module):
    def __init__(self, nb_classes=10):
        super(VGG16, self).__init__()
        self.vgg16 = models.vgg16_bn()
        self.linear = nn.Linear(1000, nb_classes)
        
    def forward(self, x):
        x = self.vgg16(x)
        x = self.linear(x)
        return x

model = VGG16()
chkpt =  torch.load("Model_6") 
model.load_state_dict(chkpt['model_state_dict'], strict= False)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [6]:
img_size = 224
test_transform = transforms.Compose([
                     transforms.Resize(size=224),
                     transforms.ToTensor(),
                     transforms.Normalize((0.5, 0.5, 0.5), (1, 1, 1)),
                     ])
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
np.swapaxes(testset.data, 1, 3).astype(np.float32)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=8)

Files already downloaded and verified


In [17]:

classifier = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(224, 224, 3),
    nb_classes=10,
)

leny = 0
accuracy = 0
# Evaluate the ART classifier on benign test examples
for idx, (data, target) in enumerate(testloader):
    x = data.numpy()
    y = target.numpy()
    predictions = classifier.predict(x)
    leny += len(y)
    accuracy += np.count_nonzero(np.argmax(predictions, axis=1) == y )
accuracy = accuracy/leny

print("Accuracy on benign test examples: {}%".format(accuracy * 100))


Accuracy on benign test examples: 90.95%


In [ ]:
# Step 6: Generate adversarial test examples
attack = FastGradientMethod(estimator=classifier, eps=0.01)
leny = 0
accuracy = 0
for idx, (data, target) in enumerate(testloader):
    y = target.numpy()
    x_test_adv = attack.generate(x=data.numpy())
    predictions = classifier.predict(x_test_adv, 100)
    leny += len(y)
    accuracy += np.count_nonzero(np.argmax(predictions, axis=1) == y )
    # print("in loop", accuracy)

accuracy = accuracy/leny
print("final accuracy", accuracy)